In [ ]:
import numpy as np

import ipywidgets as w

import bqplot.pyplot as plt
from bqplot import *

In [ ]:
n_slider = w.IntSlider(description='Points', min=1000, value=2000, max=5000, step=200)
n_slider.layout.width = '400px'

go_btn = w.Button(description='GO', button_style='success')

pi_label_tmpl = '<div style="font-size: 20px; font-weight: bold;">pi = {pi:.4f} for n = {n}</div>'
pi_label = w.HTML(pi_label_tmpl.format(pi=0, n=0), layout=w.Layout(margin='60px 0px 0px 0px'))

xs, ys = LinearScale(min=0, max=1), LinearScale(min=0, max=1)

# points inside arc
common_args = dict(x=[], y=[], 
                   stroke='black', 
                   scales={'x': xs, 'y': ys}, 
                   default_size=20)
in_dots = ScatterGL(**common_args, colors=['limegreen'])

# points outside arc
out_dots = ScatterGL(**common_args, colors=['orangered'])

xax = Axis(scale=xs, grid_lines='none', num_ticks=2)
yax = Axis(scale=ys, orientation='vertical', grid_lines='none', num_ticks=2)

# draw quadrant arc
arc_x = np.linspace(0, 1, 200)
arc_y = np.sqrt(1 - arc_x ** 2)
arc = plt.plot(arc_x, arc_y, 'g', stroke_width=1)

fig = Figure(marks=[arc, in_dots, out_dots], axes=[xax, yax], title='Estimation Of pi')
fig.layout.width = '440px'
fig.layout.height = '400px'

def animate(*args):
    N = n_slider.value

    # simulate X, Y ~ U[0, 1]
    x, y = np.random.rand(2, N)
    
    x_in, y_in, x_out, y_out = [], [], [], []
    for i in range(N):
        if x[i] ** 2 + y[i] ** 2 < 1:
            x_in.append(x[i])
            y_in.append(y[i])
            
            with in_dots.hold_sync():
                in_dots.x = x_in
                in_dots.y = y_in
        else:
            x_out.append(x[i])
            y_out.append(y[i])
            
            with out_dots.hold_sync():
                out_dots.x = x_out
                out_dots.y = y_out

        if i > 0:
            pi = 4 * len(x_in) / i
        else:
            pi = 0
        
        pi_label.value = pi_label_tmpl.format(pi=pi, n=i+1)

go_btn.on_click(animate)

w.VBox([w.HBox([n_slider, go_btn]), 
        w.HBox([fig, pi_label])
       ])